### Text Mining Homework 6

 Among the entire data of IMDB movie review data (IMDB Dataset.csv), the similarity between the three words and the review document are analyzed by using only review of the first 1000 data after preprocessing procedures as follows
 
A. cat
B. hero
C. action

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

data = pd.read_csv('../Data/IMDBDataset.csv', engine = 'python')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [2]:
n = 1000
reviews = []
for row in data['review'][0:n] : reviews.append(BeautifulSoup(row, 'html5lib').get_text())

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_54168/606716564.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  for row in data['review'][0:n] : reviews.append(BeautifulSoup(row, 'html5lib').get_text())


In [3]:
review_list = []
for row1 in reviews : review_list.append(re.sub('[^a-zA-Z]',' ', row1).lower())

#### Problem 1

Use `word2vec` (5-gram, CBOW, number of hidden vector elements = 100, minimum word frequency = 5, number of learning processes = 4) from the genism package to report the word most similar to the above three words.

In [4]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences = review_list, 
    vector_size = 100, 
    window = 5, 
    min_count = 5, 
    workers = 4, 
    sg = 0
)

above_words = ['cat','hero','action']

In [5]:
similar_word = model.wv.most_similar(positive = above_words)
print(f"The word most similar to {above_words} is {similar_word[0][0]} with a similarity score of {similar_word[0][1]}")

KeyError: "Key 'cat' not present in vocabulary"

#### Probelm 2

Using `doc2vec` (vector_size = 300, alpha = 0.025, min_alpha = 0.025, workers = 8, window = 8) of the genism package (learning with epoch = 50) to find the document most similar to the first document.

In [6]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

model = Doc2Vec(
    vector_size = 300, 
    alpha = 0.025, 
    min_alpha = 0.025, 
    workers = 8, 
    window = 8, 
    epochs = 50
)

In [7]:
tagged_data = [TaggedDocument(words = _d, tags = [str(i)]) for i, _d in enumerate(review_list)]
model.build_vocab(tagged_data)

In [8]:
model.train(
    tagged_data, 
    total_examples = model.corpus_count, 
    epochs = model.epochs
)

In [9]:
first_doc_vec = model.infer_vector(review_list)

In [10]:
similar_docs = model.dv.most_similar([first_doc_vec], topn = 1)
print(f"Document most similar to the first document is {similar_docs[0][0]}'s document whose similarity is {similar_docs[0][1] :.4f}")

Document most similar to the first document is 296's document whose similarity is 0.1611


#### Problem 3

Report `doc2vec` of genism package as the most similar document to IMDB Movie Review 1st document

In [11]:
tagged_documents = [TaggedDocument(words=doc.split(), tags = [str(i)]) for i, doc in enumerate(review_list)]

model = Doc2Vec(
    vector_size = 100, 
    window = 8, 
    min_count = 2, 
    workers = 8, 
    epochs = 40
)
model.build_vocab(tagged_documents)
model.train(
    tagged_documents, 
    total_examples = model.corpus_count, 
    epochs = model.epochs
)

In [12]:
vec_first_document = model.infer_vector(review_list)
similar_documents = model.dv.most_similar([vec_first_document], topn = 1)
print(f"Document most similar to the first document: {similar_documents[0]}")

Document most similar to the first document: ('861', 0.2421104460954666)
